In [1]:
import json
import logging
import os

from dotenv import load_dotenv
from openai import OpenAI
from pymongo import MongoClient, UpdateOne
from rich.logging import RichHandler

from bson.objectid import ObjectId
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)

load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
Database_Name = os.getenv("DATABASE")
edps_claims_collection = os.getenv("EDPS_CLAIMS_COLLECTION")
pharmacy_claims_collection = os.getenv("PHARMACY_CLAIMS_COLLECTION")
eligibility_collection = os.getenv("ELIGIBILITY_COLLECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
mbi_crosswalk_collection = os.getenv("MBI_CROSSWALK_COLLECTION")

LLM_MODEL = "gpt-4o-mini"
llm_client = OpenAI(api_key=OPENAI_API_KEY)

MBI_CROSSWALK_MAP = {}


In [2]:
def get_mbi_crosswalk_map():
    """Fetch MemberID -> MBI mapping from Mongodb and return as dict."""
    try:
        with MongoClient(MONGODB_URI) as client:
            db = client[Database_Name]
            projection = {"_id": 0, "created_dt": 0}
            cursor = db[mbi_crosswalk_collection].find({}, projection)

            crosswalk_map = {
                doc.get("MemberID"): doc.get("MBI")
                for doc in cursor
                if doc.get("MemberID") is not None and doc.get("MBI") is not None
            }

            return crosswalk_map

    except Exception as e:
        logging.error(f"Failed to load MBI crosswalk map: {e}")
        return {}


In [3]:
logging.info("Loading MBI crosswalk map into memory...")
MBI_CROSSWALK_MAP = get_mbi_crosswalk_map()
logging.info(f"Loaded {len(MBI_CROSSWALK_MAP)} crosswalk records.")

[18:01:22] INFO     Loading MBI crosswalk map into memory...                                        ]8;id=958414;file:///tmp/ipykernel_108803/2425621955.py\2425621955.py]8;;\:]8;id=443794;file:///tmp/ipykernel_108803/2425621955.py#1\1]8;;\

           INFO     You appear to be connected to a CosmosDB cluster. For more information regarding  ]8;id=227899;file:///home/linux-dex/Documents/HDM/logistic_regression/.venv/lib/python3.13/site-packages/pymongo/logger.py\logger.py]8;;\:]8;id=753812;file:///home/linux-dex/Documents/HDM/logistic_regression/.venv/lib/python3.13/site-packages/pymongo/logger.py#121\121]8;;\
                    feature compatibility and support please visit                                                 
                    https://www.mongodb.com/supportability/cosmosdb                                                

[18:01:25] INFO     You appear to be connected to a CosmosDB cluster. For more information regarding  ]8;id=520126;file:///home/linux-dex/Documents/HDM/logistic_regression/.venv/lib/python3.13/site-packages/pymongo/logger.py\logger.py]8;;\:]8;id=579572;file:///home/linux-dex/Documents/HDM/logistic_regression/.venv/lib/python3.13/site-packages/pymongo/logger.py#121\121]8;;\
                    feature compatibility and support please visit                                                 
                    https://www.mongodb.com/supportability/cosmosdb                                                

[18:01:34] INFO     Loaded 34327 crosswalk records.                                                 ]8;id=639904;file:///tmp/ipykernel_108803/2425621955.py\2425621955.py]8;;\:]8;id=769248;file:///tmp/ipykernel_108803/2425621955.py#3\3]8;;\

In [ ]:
def load_members_with_claims_from_docs(eligibility_docs):
    """Given eligibility documents, fetch medical & pharmacy claims using MBI if available."""
    
    global MBI_CROSSWALK_MAP

    try:
        client = MongoClient(MONGODB_URI)
        db = client[Database_Name]
    except Exception as e:
        logging.error(f"Error connecting to MongoDB: {e}")
        return []

    members = []

    for el in eligibility_docs:
        member_id = el["memberId"]

        # MBI fallback logic
        mbi_lookup_id = MBI_CROSSWALK_MAP.get(member_id, member_id)

        # ---- Medical Claims (EDPS) ----
        medical_claims_cursor = db[edps_claims_collection].find(
            {"Member.Subscriber_ID": mbi_lookup_id},
            {
                "_id": 0,
                "Diagnosis.Diag_Codes": 1,
                "ServiceLine.LXServiceNo": 1,
                "ServiceLine.BilledCPT_Code": 1,
                "ServiceLine.BilledCPTDesc": 1,
                "ServiceLine.Line_SvcDate": 1,
                "Claim.ClaimID": 1,
                "Claim.POS": 1,
                "Type_of_Bill": 1,
                "Provider.BillProv_NPI": 1,
                "Provider.BillProv_LastName": 1,
                "Member.Subscriber_ID": 1,
                "Member.Subscriber_DOB": 1,
                "Member.Subscriber_Gender": 1,
            },
        )
        medical_claims = list(medical_claims_cursor)

        # ---- Pharmacy Claims ----
        pharmacy_claims_cursor = db[pharmacy_claims_collection].find(
            {"Member ID": member_id},
            {
                "_id": 0,
                "Member ID": 1,
                "NDC": 1,
                "Product Label Name": 1,
                "Fill Date": 1,
                "Days Supply": 1,
                "Metric Quantity": 1,
                "Prescriber ID": 1,
                "Prescriber Name": 1,
                "Total Billed": 1,
            },
        )

        pharmacy_claims = [
            {
                "memberId": pc.get("Member ID"),
                "ndc": str(pc.get("NDC")),
                "drugName": pc.get("Product Label Name"),
                "fillDate": (
                    pc.get("Fill Date").strftime("%Y-%m-%d")
                    if pc.get("Fill Date")
                    else None
                ),
                "daysSupply": pc.get("Days Supply"),
                "quantityDispensed": pc.get("Metric Quantity"),
                "prescriberNPI": pc.get("Prescriber ID"),
                "prescriberName": pc.get("Prescriber Name"),
                "totalBilled": pc.get("Total Billed"),
            }
            for pc in pharmacy_claims_cursor
        ]

        members.append(
            {
                "memberId": member_id,
                "eligibility": el,
                "medicalClaims": medical_claims,
                "pharmacyClaims": pharmacy_claims,
            }
        )

    return members